### Operational Model 

In [69]:
%load_ext autoreload
%autoreload 2

import numpy as np
import random
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from xgboost import XGBClassifier
import os
from sklearn.feature_selection import RFECV
import pandas as pd
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    StratifiedKFold,
    train_test_split,
    KFold)
import numpy as np
from numpy.lib.function_base import average
import pandas as pd
import matplotlib.pyplot as plt
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import (
    mean_absolute_error, 
    mean_squared_error,
    recall_score,
    f1_score,
    precision_score,
    confusion_matrix,
    make_scorer)
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
 
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
import importlib
import os
from sklearn.feature_selection import (
    SelectKBest,
    SequentialFeatureSelector,
    RFE,
    mutual_info_regression,
    f_regression,
    mutual_info_classif)
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.inspection import permutation_importance
import xgboost as xgb
import pickle
import openpyxl
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
import geopandas as gpd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [70]:
# Setting directory
wor_dir="/home/fbf/"


wor_dir="C:/Typhoon_IBF/Typhoon-Impact-based-forecasting-model/IBF-Typhoon-model"

os.chdir(wor_dir)

cdir = os.getcwd()

### Loading the data

In [71]:
combined_input_data=pd.read_csv("data/model_input/df_modelinput_july.csv") 
tphoon_events=combined_input_data[['typhoon','DAM_perc_dmg']].groupby('typhoon').size().to_dict()
### for hyper parameter optimization we looped over typhoon events with at least 100 data entries  
typhoons_with_impact_data= [key for key,value in tphoon_events.items() if value>100]
#combined_input_data=combined_input_data[combined_input_data.typhoon.isin(typhoons_with_impact_data)]

In [72]:
def set_zeros(x):
    x_max = 25
    y_max = 50
    
    v_max = x[0]
    rainfall_max = x[1]
    damage = x[2]
    if pd.notnull(damage):
        value = damage
    elif v_max > x_max or rainfall_max > y_max:
        value =damage
    elif (v_max < np.sqrt((1- (rainfall_max**2/y_max ** 2))*x_max ** 2)):
        value = 0
    #elif ((v_max < x_max)  and  (rainfall_max_6h < y_max) ):
    #elif (v_max < x_max ):
    #value = 0
    else:
        value = np.nan

    return value

combined_input_data["DAM_perc_dmg"] = combined_input_data[["HAZ_v_max", "HAZ_rainfall_Total", "DAM_perc_dmg"]].apply(set_zeros, axis="columns")

### result of parameter optimization 

In [73]:
selected_params_xgb_regr={'colsample_bytree': 1.0,
                          'eta': 0.05,
                          'gamma': 1.0,
                          'learning_rate': 0.025,
                          'max_depth': 8,
                          'min_child_weight': 1.0,
                          'n_estimators': 80.0,
                          'early_stopping_rounds ':10,
                          'subsample': 0.8
                          }
wor_dir="C:/Typhoon_IBF/Typhoon-Impact-based-forecasting-model/IBF-Typhoon-model"
file_name = "models/output/v2/selected_params_xgb_regr.p"
 

In [ ]:
# split data into train and test sets

SEED = 314159265
 
test_size = 0.1

# Full dataset for feature selection

combined_input_data_ = combined_input_data[combined_input_data['DAM_perc_dmg'].notnull()]
 

selected_features_xgb_regr= ['HAZ_rainfall_max_24h','HAZ_v_max',#'HAZ_dis_track_min',
                             'TOP_mean_slope',
                                       'TOP_mean_elevation_m','TOP_ruggedness_stdev','TOP_mean_ruggedness','TOP_slope_stdev',
                                       'VUL_poverty_perc','GEN_with_coast','VUL_Housing_Units','VUL_StrongRoof_StrongWall',
                                       'VUL_StrongRoof_LightWall','VUL_StrongRoof_SalvageWall','VUL_LightRoof_StrongWall',
                                       'VUL_LightRoof_LightWall','VUL_SalvagedRoof_StrongWall','VUL_SalvagedRoof_LightWall',
                                       'VUL_SalvagedRoof_SalvageWall','VUL_vulnerable_groups','VUL_pantawid_pamilya_beneficiary']

selected_features_xgb_regr_all_event= ['HAZ_v_max',#'HAZ_dis_track_min',
                                       'TOP_mean_slope',
                                       'TOP_mean_elevation_m','TOP_ruggedness_stdev','TOP_mean_ruggedness','TOP_slope_stdev',
                                       'VUL_poverty_perc','GEN_with_coast','VUL_Housing_Units','VUL_StrongRoof_StrongWall',
                                       'VUL_StrongRoof_LightWall','VUL_StrongRoof_SalvageWall','VUL_LightRoof_StrongWall',
                                       'VUL_LightRoof_LightWall','VUL_SalvagedRoof_StrongWall','VUL_SalvagedRoof_LightWall',
                                       'VUL_SalvagedRoof_SalvageWall','VUL_vulnerable_groups','VUL_pantawid_pamilya_beneficiary']

X = combined_input_data_[selected_features_xgb_regr]

y = combined_input_data_["DAM_perc_dmg"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=SEED)

# fit model no training data
 
from sklearn.metrics import mean_squared_error,mean_absolute_error

#selected_features_xgb_regr_all_event
reg = xgb.XGBRegressor(base_score=0.5,
             booster='gbtree', 
             subsample=0.8,
             eta=0.05,
             max_depth=8, 
             colsample_bylevel=1,
             colsample_bynode=1, 
             colsample_bytree=1,
             early_stopping_rounds=10, 
             eval_metric=mean_absolute_error,
             gamma=1, 
             objective='reg:squarederror',
             gpu_id=-1, 
             grow_policy='depthwise', 
             learning_rate=0.025,
             min_child_weight=1,
             n_estimators=200,        
             random_state=42,
             tree_method="hist",
             )


eval_set=[(X_train, y_train), ( X_test, y_test)]
reg.fit(X, y, eval_set=eval_set)
 
y_pred = reg.predict(X_test)

In [ ]:
y_pred = reg.predict(X)
y_pred[y_pred < 0] = 0
combined_input_data_['DMG_predicted']=y_pred

### Aggregated impact, number of completely damaged houses per typhoon event 

In [ ]:

def division(x, y):
    try:
        value = int((x * y)/100)
        
    except:
        value = np.nan
    
    return x if value>x else value  

f_path = os.path.join(wor_dir,"data/all_predisaster_indicators.csv")
 
df_predisasters = pd.read_csv(f_path)

df_ = pd.merge(combined_input_data_[['Mun_Code','typhoon','DAM_perc_dmg','DMG_predicted']], df_predisasters[['Housing Units','Mun_Code']],  how='left', left_on='Mun_Code', right_on = 'Mun_Code') 

df_["number_dmg_Build_prediction"] = df_.apply(lambda x: division(x["Housing Units"], x["DMG_predicted"]), axis=1).values
df_["number_dmg_Build_dromic"] = df_.apply(lambda x: division(x["Housing Units"], x["DAM_perc_dmg"]), axis=1).values

df_im_haz=df_.groupby('typhoon').agg(NUmber_of_affected_municipality=('Mun_Code','count'),
                                     average_ML_Model=('DMG_predicted', average),
                                     average_dromic=('DAM_perc_dmg', average),
                                     Total_buildings_ML_Model=('number_dmg_Build_prediction', sum),
                                     Total_buildings_dromic=('number_dmg_Build_dromic', sum)).sort_values(by='Total_buildings_dromic',ascending=False).reset_index()

 